In [1]:
import warnings
import time
import sys
import os
import datetime
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from model_zoo import my_lgb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import pickle as pkl
%matplotlib inline

In [2]:
# 评分函数与提交函数
def score(pre, truth):
    return 1 / (MAE(pre, truth) + 1)

def MAE(pre, truth):
    return abs((np.rint(pre) - truth)).mean()

def submit(model_name='default', predictions=None):
    sub_df = pd.read_csv('dataset/submit_example.csv')
    sub_df[' score'] = np.rint(predictions).astype(int)
    sub_df.sort_values('score', inplace=True)
    
    sub_df.to_csv("output/{}.csv".format(model_name), index=False)

In [3]:
train_df = pd.read_csv('dataset/train_dataset.csv')
test_df = pd.read_csv('dataset/test_dataset.csv')

特征工程，简单粗暴。

* 长尾数据处理

* 过拟合

In [ ]:
def percent(df, feature):
    for f in feature:
        data[f] = np.rint(data[f])
        new_feature = f + '_count'
        tmp = df.groupby(f).size().reset_index().rename(columns={0:new_feature})
        #  tmp[new_feature] = tmp[new_feature].cumsum()*100/df.shape[0]
        df = df.merge(tmp, 'left', on = f)
    return df
    
def generate_feature(df):
    df['用户前五个月平均消费值（元）'] = (df['用户近6个月平均消费值（元）']*6 - df['用户账单当月总费用（元）'])/5
    df['当月消费值较前五个月平均消费值'] = df['用户账单当月总费用（元）'] - df['用户前五个月平均消费值（元）']
    
    app_col = ['当月视频播放类应用使用次数','当月金融理财类应用使用总次数','当月网购类应用使用次数']
    df['当月网购类应用使用次数' + '百分比'] = (df['当月网购类应用使用次数'])/(df[app_col].sum(axis=1) + 1)
    
    df.loc[df['用户年龄']==0, '用户年龄'] = df['用户年龄'].mode()
    
    long_tail = ['当月视频播放类应用使用次数', '当月金融理财类应用使用总次数','当月网购类应用使用次数',
                 '缴费用户最近一次缴费金额（元）','用户当月账户余额（元）', '用户账单当月总费用（元）',
                 '用户近6个月平均消费值（元）','当月通话交往圈人数','当月物流快递类应用使用次数',
                '当月飞机类应用使用次数','当月火车类应用使用次数', '当月旅游资讯类应用使用次数'] 
    for feature in long_tail:
        threshold = int(df[feature].std()*3 + df[feature].mean())
        df[feature] = df[feature].apply(lambda x: x if x < threshold else threshold)
    
    df['用户最近一次缴费距今时长（月）'] = df['缴费用户最近一次缴费金额（元）'].apply(lambda x: 1 if x>0 else 0)

    return df

In [37]:
target = train_df['信用分']
data = pd.concat([train_df.drop(columns=['信用分']), test_df], axis=0, ignore_index=True)
data = generate_feature(data)
train = data.loc[:49999, :]
test = data.loc[50000:, :]

In [38]:
drop_columns = ['用户编码','是否大学生客户','用户实名制是否通过核实',
                '当月是否逛过福州仓山万达', '当月是否到过福州山姆会员店']
X_train = train.drop(columns=drop_columns).values
y_train = target.values
X_test = test.drop(columns=drop_columns).values

param = {'num_leaves': 30,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.008,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 0.15,
         "lambda_l2": 0.04,
         "verbosity": -1}

param = {'num_leaves': 35,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.004,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 0.15,
         "lambda_l2": 0.05,
         "verbosity": -1}
 94

阴差阳错我用的都是mse,只是用了不同的随机种子

In [39]:
def cache_save():
    with open('output/cache/params.pkl', 'wb') as f:
        pkl.dump(best_params, f)
    with open('output/cache/valid.pkl', 'wb') as f:
        pkl.dump(valid, f)
    with open('output/cache/res.pkl', 'wb') as f:
        pkl.dump(res, f)

In [42]:
param1 = {'num_leaves': 35,
         'objective':'regression_l2',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 1,
         "verbosity": -1}
param2 = {'num_leaves': 35,
         'objective':'regression_l1',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 1,
         "verbosity": -1}
valid = []
res = []
best_params = {}
for seed in [2018,2019]:
    print('fire!!!!!')
    print('*'*50)
    start = time.time()
    clf = my_lgb(folds=10, seed=seed)
    _best_params = clf.optimize_lgb(X_train, y_train, X_test, param1)
    clf.inference_folds(X_train, y_train, X_test, _best_params)
    
    best_params['{}-mse'.format(seed)] = _best_params
    valid.append(clf.oof)
    res.append(clf.results)
    cache_save()
    del clf
    print('*'*25)
    
    clf = my_lgb(folds=10, seed=seed)
    _best_params = clf.optimize_lgb(X_train, y_train, X_test, param2)
    clf.inference_folds(X_train, y_train, X_test, _best_params)
    best_params['{}-mae'.format(seed)] = _best_params

    valid.append(clf.oof)
    res.append(clf.results)
    cache_save()
    del clf
    print('*'*25)
    print('seed:{},training spend {}s'.format(seed, time.time()-start))
    print('*'*50)

fire!!!!!
**************************************************
|   iter    |  target   | lambda_l1 | lambda_l2 | num_le... |
-------------------------------------------------------------
fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.791	valid_1's l1: 20.8974
[400]	training's l1: 16.5015	valid_1's l1: 16.8072
[600]	training's l1: 15.1857	valid_1's l1: 15.6485
[800]	training's l1: 14.7228	valid_1's l1: 15.2752
[1000]	training's l1: 14.5034	valid_1's l1: 15.125
[1200]	training's l1: 14.3682	valid_1's l1: 15.0578
[1400]	training's l1: 14.2666	valid_1's l1: 15.0215
[1600]	training's l1: 14.1809	valid_1's l1: 14.9988
[1800]	training's l1: 14.1055	valid_1's l1: 14.9827
[2000]	training's l1: 14.034	valid_1's l1: 14.9683
[2200]	training's l1: 13.9677	valid_1's l1: 14.962
[2400]	training's l1: 13.9044	valid_1's l1: 14.9532
[2600]	training's l1: 13.8426	valid_1's l1: 14.9454
[2800]	training's l1: 13.7837	valid_1's l1: 14.9436
Early stopping, best it

[2000]	training's l1: 14.0696	valid_1's l1: 14.6679
[2200]	training's l1: 14.0031	valid_1's l1: 14.6575
[2400]	training's l1: 13.9392	valid_1's l1: 14.6492
[2600]	training's l1: 13.8785	valid_1's l1: 14.6414
[2800]	training's l1: 13.8203	valid_1's l1: 14.6337
[3000]	training's l1: 13.7648	valid_1's l1: 14.6281
[3200]	training's l1: 13.7098	valid_1's l1: 14.6242
[3400]	training's l1: 13.6567	valid_1's l1: 14.6189
[3600]	training's l1: 13.6037	valid_1's l1: 14.6173
[3800]	training's l1: 13.5515	valid_1's l1: 14.6127
[4000]	training's l1: 13.5003	valid_1's l1: 14.608
[4200]	training's l1: 13.45	valid_1's l1: 14.6059
Early stopping, best iteration is:
[4126]	training's l1: 13.468	valid_1's l1: 14.6055
fold n°9
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.8174	valid_1's l1: 20.7937
[400]	training's l1: 16.5485	valid_1's l1: 16.5054
[600]	training's l1: 15.234	valid_1's l1: 15.2564
[800]	training's l1: 14.7687	valid_1's l1: 14.8359
[1000]	training's 

[1600]	training's l1: 14.1415	valid_1's l1: 14.9571
[1800]	training's l1: 14.0632	valid_1's l1: 14.9367
[2000]	training's l1: 13.9897	valid_1's l1: 14.9223
[2200]	training's l1: 13.9222	valid_1's l1: 14.9102
[2400]	training's l1: 13.8575	valid_1's l1: 14.9013
[2600]	training's l1: 13.794	valid_1's l1: 14.8919
[2800]	training's l1: 13.734	valid_1's l1: 14.885
[3000]	training's l1: 13.6747	valid_1's l1: 14.875
[3200]	training's l1: 13.6181	valid_1's l1: 14.8687
[3400]	training's l1: 13.5621	valid_1's l1: 14.8653
[3600]	training's l1: 13.5061	valid_1's l1: 14.8617
[3800]	training's l1: 13.4515	valid_1's l1: 14.852
[4000]	training's l1: 13.3985	valid_1's l1: 14.8487
[4200]	training's l1: 13.3453	valid_1's l1: 14.8437
[4400]	training's l1: 13.2932	valid_1's l1: 14.8387
[4600]	training's l1: 13.2416	valid_1's l1: 14.8348
[4800]	training's l1: 13.193	valid_1's l1: 14.834
[5000]	training's l1: 13.1433	valid_1's l1: 14.8317
Early stopping, best iteration is:
[5051]	training's l1: 13.1307	valid_

Early stopping, best iteration is:
[3422]	training's l1: 13.4193	valid_1's l1: 14.6085
fold n°4
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.6736	valid_1's l1: 21.1579
[400]	training's l1: 16.4141	valid_1's l1: 16.6712
[600]	training's l1: 15.1173	valid_1's l1: 15.3066
[800]	training's l1: 14.6589	valid_1's l1: 14.8665
[1000]	training's l1: 14.4334	valid_1's l1: 14.6884
[1200]	training's l1: 14.2895	valid_1's l1: 14.6012
[1400]	training's l1: 14.1761	valid_1's l1: 14.5546
[1600]	training's l1: 14.0807	valid_1's l1: 14.5255
[1800]	training's l1: 13.9941	valid_1's l1: 14.5064
[2000]	training's l1: 13.9132	valid_1's l1: 14.4888
[2200]	training's l1: 13.8375	valid_1's l1: 14.4776
[2400]	training's l1: 13.7641	valid_1's l1: 14.4689
[2600]	training's l1: 13.6952	valid_1's l1: 14.4612
Early stopping, best iteration is:
[2689]	training's l1: 13.6651	valid_1's l1: 14.459
fold n°5
Training until validation scores don't improve for 100 rounds.
[200]	train

[1400]	training's l1: 14.2007	valid_1's l1: 15.029
[1600]	training's l1: 14.1093	valid_1's l1: 15.0028
[1800]	training's l1: 14.0287	valid_1's l1: 14.9827
[2000]	training's l1: 13.9525	valid_1's l1: 14.9665
[2200]	training's l1: 13.881	valid_1's l1: 14.9576
[2400]	training's l1: 13.8133	valid_1's l1: 14.9509
[2600]	training's l1: 13.7473	valid_1's l1: 14.9427
[2800]	training's l1: 13.6848	valid_1's l1: 14.9377
[3000]	training's l1: 13.623	valid_1's l1: 14.9328
[3200]	training's l1: 13.5634	valid_1's l1: 14.9309
[3400]	training's l1: 13.5045	valid_1's l1: 14.9253
[3600]	training's l1: 13.4473	valid_1's l1: 14.9217
[3800]	training's l1: 13.3915	valid_1's l1: 14.9194
[4000]	training's l1: 13.3366	valid_1's l1: 14.9165
Early stopping, best iteration is:
[4082]	training's l1: 13.3138	valid_1's l1: 14.9144
fold n°3
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.7418	valid_1's l1: 20.7415
[400]	training's l1: 16.4737	valid_1's l1: 16.5202
[600]	training

...

fold n°9
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.7973	valid_1's l1: 20.6929
[400]	training's l1: 16.7003	valid_1's l1: 16.9492
[600]	training's l1: 15.3159	valid_1's l1: 15.7248
[800]	training's l1: 14.7812	valid_1's l1: 15.2901
[1000]	training's l1: 14.519	valid_1's l1: 15.1086
[1200]	training's l1: 14.3569	valid_1's l1: 15.0181
[1400]	training's l1: 14.2408	valid_1's l1: 14.9702
[1600]	training's l1: 14.1436	valid_1's l1: 14.9378
[1800]	training's l1: 14.0591	valid_1's l1: 14.9175
[2000]	training's l1: 13.9837	valid_1's l1: 14.9
[2200]	training's l1: 13.9136	valid_1's l1: 14.8876
[2400]	training's l1: 13.8465	valid_1's l1: 14.8737
[2600]	training's l1: 13.7828	valid_1's l1: 14.8619
[2800]	training's l1: 13.7229	valid_1's l1: 14.8548
[3000]	training's l1: 13.666	valid_1's l1: 14.8502
[3200]	training's l1: 13.61	valid_1's l1: 14.8441
[3400]	training's l1: 13.5561	valid_1's l1: 14.8402
Early stopping, best iteration is:
[3478]	training's l1

[2400]	training's l1: 13.6712	valid_1's l1: 14.8083
[2600]	training's l1: 13.5999	valid_1's l1: 14.8047
[2800]	training's l1: 13.533	valid_1's l1: 14.8028
[3000]	training's l1: 13.4687	valid_1's l1: 14.7997
Early stopping, best iteration is:
[2969]	training's l1: 13.4787	valid_1's l1: 14.7987
fold n°7
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.7164	valid_1's l1: 21.1614
[400]	training's l1: 16.6492	valid_1's l1: 17.045
[600]	training's l1: 15.2719	valid_1's l1: 15.6841
[800]	training's l1: 14.7219	valid_1's l1: 15.192
[1000]	training's l1: 14.4415	valid_1's l1: 14.9971
[1200]	training's l1: 14.2649	valid_1's l1: 14.8985
[1400]	training's l1: 14.133	valid_1's l1: 14.8403
[1600]	training's l1: 14.0239	valid_1's l1: 14.8053
[1800]	training's l1: 13.9274	valid_1's l1: 14.7758
[2000]	training's l1: 13.8395	valid_1's l1: 14.7555
[2200]	training's l1: 13.7594	valid_1's l1: 14.7437
[2400]	training's l1: 13.683	valid_1's l1: 14.7318
[2600]	training's 

[3400]	training's l1: 13.612	valid_1's l1: 14.2973
[3600]	training's l1: 13.5609	valid_1's l1: 14.2906
[3800]	training's l1: 13.5111	valid_1's l1: 14.2883
[4000]	training's l1: 13.4621	valid_1's l1: 14.2861
[4200]	training's l1: 13.4153	valid_1's l1: 14.2812
[4400]	training's l1: 13.3689	valid_1's l1: 14.2788
[4600]	training's l1: 13.3239	valid_1's l1: 14.2751
Early stopping, best iteration is:
[4506]	training's l1: 13.345	valid_1's l1: 14.275
fold n°4
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.7858	valid_1's l1: 20.9693
[400]	training's l1: 16.7111	valid_1's l1: 16.9871
[600]	training's l1: 15.3403	valid_1's l1: 15.6589
[800]	training's l1: 14.8039	valid_1's l1: 15.1629
[1000]	training's l1: 14.5444	valid_1's l1: 14.943
[1200]	training's l1: 14.3825	valid_1's l1: 14.8263
[1400]	training's l1: 14.2644	valid_1's l1: 14.7624
[1600]	training's l1: 14.167	valid_1's l1: 14.7224
[1800]	training's l1: 14.082	valid_1's l1: 14.7001
[2000]	training's l

[4200]	training's l1: 13.3922	valid_1's l1: 14.4892
[4400]	training's l1: 13.3464	valid_1's l1: 14.4868
[4600]	training's l1: 13.3012	valid_1's l1: 14.4852
[4800]	training's l1: 13.2589	valid_1's l1: 14.4831
Early stopping, best iteration is:
[4815]	training's l1: 13.2555	valid_1's l1: 14.4819
score: 0.06394197, MAE: 14.63918
|  15       |  0.06394  |  5.007    |  2.488    |  30.06    |
Final result:{'target': 0.06396683959035636, 'params': {'lambda_l1': 6.236126324840814, 'lambda_l2': 1.1317532148405545, 'num_leaves': 32.606148600406456}}, spend -3700.7271332740784s
fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.7756	valid_1's l1: 20.8138
[400]	training's l1: 16.6851	valid_1's l1: 16.8167
[600]	training's l1: 15.3083	valid_1's l1: 15.5494
[800]	training's l1: 14.7747	valid_1's l1: 15.1104
[1000]	training's l1: 14.5116	valid_1's l1: 14.9157
[1200]	training's l1: 14.3473	valid_1's l1: 14.8188
[1400]	training's l1: 14.2265	valid_1's l1: 14

[2800]	training's l1: 13.6856	valid_1's l1: 14.7022
[3000]	training's l1: 13.6287	valid_1's l1: 14.698
[3200]	training's l1: 13.5729	valid_1's l1: 14.692
[3400]	training's l1: 13.5177	valid_1's l1: 14.6872
[3600]	training's l1: 13.4647	valid_1's l1: 14.6801
[3800]	training's l1: 13.4125	valid_1's l1: 14.6759
[4000]	training's l1: 13.3618	valid_1's l1: 14.6739
[4200]	training's l1: 13.3137	valid_1's l1: 14.672
Early stopping, best iteration is:
[4171]	training's l1: 13.3208	valid_1's l1: 14.6715
fold n°8
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 20.7579	valid_1's l1: 21.1215
[400]	training's l1: 16.6879	valid_1's l1: 17.0243
[600]	training's l1: 15.317	valid_1's l1: 15.64
[800]	training's l1: 14.7814	valid_1's l1: 15.1507
[1000]	training's l1: 14.5164	valid_1's l1: 14.9513
[1200]	training's l1: 14.3527	valid_1's l1: 14.8464
[1400]	training's l1: 14.2322	valid_1's l1: 14.7871
[1600]	training's l1: 14.1318	valid_1's l1: 14.7527
[1800]	training's l

In [53]:
val = valid[0]*0.25 + valid[1]*0.25 + valid[2]*0.2509 + valid[3]*0.25
score(val, y_train)

0.06416121403282744

In [55]:
final = res[0]*0.25 + res[1]*0.25 + res[2]*0.25 + res[3]*0.25
submit(model_name='313', predictions=final)

In [33]:
sub_df = pd.read_csv('dataset/submit_example.csv')
sub_df[' score'] = final
# sub_df.sort_values(' score', inplace=True)
# sub_df.to_csv("output/{}.csv".format(model_name), index=False)

In [34]:
sub_df[' score'] = sub_df[' score'].apply(lambda x: int(x) if x > 650 else int(x + 1))

In [35]:
sub_df.to_csv("output/{}.csv".format('final'), index=False)

In [ ]:
param = {'num_leaves': 31,
         'objective':'regression_l2',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.4,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity":1}
clf = my_lgb(folds=5, seed=2018)
clf.inference_folds(X_train, y_train, X_test, param)
mse_2018 = clf.oof

In [ ]:
param = {'num_leaves': 31,
         'objective':'regression_l2',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.4,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity": -1}

clf_mse = my_lgb(folds=5, seed=2019)
clf_mse.inference_folds(X_train, y_train, X_test, param)
mse_2019 = clf_mse.oof

In [ ]:
p = 0.50
q = 0.502
results = mse_2018*p+ mse_2019*q 
print(score(results, y_train))
res = clf.results*p + clf_mse.results*q
submit(model_name='model_V2_bad', predictions=res)

In [ ]:
param = {'num_leaves': 35,
         'objective':'regression_l1',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.4,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 1,
         "verbosity": -1}

clf_mae = my_lgb(folds=5, seed=2018)
clf_mae.inference_folds(X_train, y_train, X_test, param)
mae_2018 = clf_mae.oof

In [ ]:
# 0.06398976
clf_mae.submit(output_name = 'model_V1_mae_2018_1')

In [ ]:
param = {'num_leaves': 35,
         'objective':'regression_l1',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.4,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity": -1}

clf_mae_ = my_lgb(folds=5, seed=2019)
clf_mae_.inference_folds(X_train, y_train, X_test, param)
mae_2019 = clf_mae_.oof

In [ ]:
# 0.06399214
clf_mae_.submit(output_name = 'model_V1_mae_2019_1')

param = {'num_leaves': 35,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mse',
         "lambda_l1": 0.16,
         "lambda_l2": 0.1,
         "verbosity": -1}

In [ ]:
p = 0.251
q = 0.25
m = 0.25
n = 0.25
results = mse_2018*p+ mse_2019*q + mae_2018*m + mae_2019*n
print(score(results, y_train))
res = clf.results*p + clf_mse.results*q + clf_mae.results*m + clf_mae_.results*n
submit(model_name='model_V2_bad', predictions=res)

In [ ]:
residual = mae_2018 - y_train
submit(model_name='resdiual_mae', predictions=residual)

In [ ]:
# drop_columns = drop_columns.remove('信用分')
col = train.drop(columns=drop_columns).columns
f = clf_mae.importance_feature(col)
f

In [ ]:
(residual>100).sum()

In [ ]:
feature = ['用户网龄（月）', '当月通话交往圈人数', '用户年龄', 
       '用户账单当月总费用（元）', '用户近6个月平均消费值（元）', '当月金融理财类应用使用总次数', '当月视频播放类应用使用次数',
       '当月网购类应用使用次数', '用户当月账户余额（元）', '近三个月月均商场出现次数',
       '当月旅游资讯类应用使用次数', '缴费用户最近一次缴费金额（元）', '信用分']

In [ ]:
train_df.loc[residual > 100,feature].sample(10)

In [ ]:
train_df.loc[23379, :]